<a href="https://colab.research.google.com/github/SidyKing/IA_Projects/blob/main/projet_lightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJET LightGBM

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
#importation des donnees
df = pd.read_csv("/content/finance.csv")
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [6]:
#traitement des donnees

In [7]:
#verification des donnees nulles
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64

In [8]:
#verification des duplicatats
df.duplicated().sum()

0

In [9]:
#information sur les colonnes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97225 entries, 0 to 97224
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            97225 non-null  int64  
 1   type            97225 non-null  object 
 2   amount          97225 non-null  float64
 3   nameOrig        97225 non-null  object 
 4   oldbalanceOrg   97225 non-null  float64
 5   newbalanceOrig  97225 non-null  float64
 6   nameDest        97225 non-null  object 
 7   oldbalanceDest  97225 non-null  float64
 8   newbalanceDest  97224 non-null  float64
 9   isFraud         97224 non-null  float64
 10  isFlaggedFraud  97224 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 8.2+ MB


In [10]:
#suppression des types de transactions unitiles

In [11]:
#les colonnes 'isFlaggedFraud','step','nameOrig','nameDest' ne sont pas pertinentes 

df = df.drop(['isFlaggedFraud','step','nameOrig','nameDest'], axis=1)
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0.0
1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0.0
2,TRANSFER,181.00,181.0,0.00,0.0,0.0,1.0
3,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1.0
4,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0.0


In [12]:
#regardons de plus pres la colonne type(qualitatif)

In [13]:
df['type'].value_counts()

PAYMENT     38478
CASH_OUT    29839
CASH_IN     19561
TRANSFER     8371
DEBIT         976
Name: type, dtype: int64

In [14]:
df[['type','isFraud']].value_counts()

type      isFraud
PAYMENT   0.0        38477
CASH_OUT  0.0        29781
CASH_IN   0.0        19561
TRANSFER  0.0         8315
DEBIT     0.0          976
CASH_OUT  1.0           58
TRANSFER  1.0           56
dtype: int64

In [15]:
#on remarque que les types de transaction PAYMENT, CASH_IN et DEBIT ne comportent pas de transactions Frauduleuse
#on prend en compte donc que les transactions TRANSFER et CASH_OUT
X = df.loc[(df.type == 'TRANSFER') | (df.type == 'CASH_OUT')]
X

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
2,TRANSFER,181.00,181.00,0.00,0.00,0.00,1.0
3,CASH_OUT,181.00,181.00,0.00,21182.00,0.00,1.0
15,CASH_OUT,229133.94,15325.00,0.00,5083.00,51513.44,0.0
19,TRANSFER,215310.30,705.00,0.00,22425.00,0.00,0.0
24,TRANSFER,311685.89,10835.00,0.00,6267.00,2719172.89,0.0
...,...,...,...,...,...,...,...
97181,CASH_OUT,99472.32,0.00,0.00,777264.84,876737.16,0.0
97182,CASH_OUT,2695.92,6342.00,3646.08,1781392.96,1856223.23,0.0
97203,CASH_OUT,366571.04,119937.47,0.00,425199.81,662002.73,0.0
97208,CASH_OUT,52257.98,11061.00,0.00,18751.39,71009.37,0.0


In [16]:
# Binary-encoding de la colonne type
X.loc[X.type == 'TRANSFER', 'type'] = 0
X.loc[X.type == 'CASH_OUT', 'type'] = 1
X.type = X.type.astype(int)
X

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
2,0,181.00,181.00,0.00,0.00,0.00,1.0
3,1,181.00,181.00,0.00,21182.00,0.00,1.0
15,1,229133.94,15325.00,0.00,5083.00,51513.44,0.0
19,0,215310.30,705.00,0.00,22425.00,0.00,0.0
24,0,311685.89,10835.00,0.00,6267.00,2719172.89,0.0
...,...,...,...,...,...,...,...
97181,1,99472.32,0.00,0.00,777264.84,876737.16,0.0
97182,1,2695.92,6342.00,3646.08,1781392.96,1856223.23,0.0
97203,1,366571.04,119937.47,0.00,425199.81,662002.73,0.0
97208,1,52257.98,11061.00,0.00,18751.39,71009.37,0.0


In [17]:
#definition de X et Y
Y = X['isFraud']
X = X.drop('isFraud', axis=1)

In [18]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.3, random_state=12)

## K-NEAREST NEIGHBORS

In [19]:
#instanciation
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=20)
knn
#fitting du model
knn.fit(X_train,Y_train)
#prediction
pred_knn = knn.predict(X_test)
#mesure evolution
knn_acc = accuracy_score(Y_test,pred_knn)
print('le taux accuracy est de', knn_acc)

le taux accuracy est de 0.9967722236761756


## REGRESSION LOGISTIQUE

In [20]:
#instanciation
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr
#fitting du model
lr.fit(X_train,Y_train)
#prediction
pred_lr = lr.predict(X_test)
#mesure evolution
lr_acc = accuracy_score(Y_test,pred_lr)
print('accuracy de la regression logique', lr_acc)

accuracy de la regression logique 0.9990403908226467


## NAIVE BAYESIAN

In [21]:
#instanciation du model
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
#fitting du model
gnb.fit(X_train,Y_train)
#predictions du model
pred_gnb = gnb.predict(X_test)
#mesure evaluation
gnb_acc = accuracy_score(Y_test,pred_gnb)
print('accuracy du naive bayesian', gnb_acc)

accuracy du naive bayesian 0.9842100671726424


## ARBRE DE DECISION

In [22]:
#instanciation du model
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
#fitting du model
dtc.fit(X_train,Y_train)
#predictions du model
pred_dtc = dtc.predict(X_test)
#mesure evaluation
dtc_acc = accuracy_score(Y_test,pred_dtc)
print('accuracy du Decision Tree Classifier', dtc_acc)

accuracy du Decision Tree Classifier 0.9968594608741167


## LightGBM

In [23]:
#instanciation du model
import lightgbm as lgb
lgbm = lgb.LGBMClassifier()
#fitting du model
lgbm.fit(X_train,Y_train)
#predictions du model
pred_lgbm = lgbm.predict(X_test)
#mesure evaluation
lgbm_acc = accuracy_score(Y_test,pred_lgbm)
print('accuracy du LightGBM', lgbm_acc)

accuracy du LightGBM 0.9966849864782343


In [24]:
#optimisons le model lightGBM
from sklearn.model_selection import GridSearchCV
#parametre du grid
params=[
    {'boosting_type':['gbdt','dart','goss','rf']},
    {'num_leaves':[31,10,15,50,40,31,100,500]},
    {'max_depth':[-1,10,55,15,30,100]},
    {'learning_rate':[0.1,0.2,1.0,10]},
    {'n_estimators':[100,50,25,15,500]},
    {'subsample_for_bin':[200000,300000,500000,100000]},
    {'objective':[None,'binary','multiclass']},
    {'class_weight':[None,'balanced']},
    {'min_split_gain':[0.0,0.1,0.2,0.7]},
    {'min_child_weight':[0.001,0.002,0.100,0.010,0.03]},
    {'min_child_samples':[20,10,50,5,100]},
    {'subsample':[1.0,0.1,2.0,1.2,5.0,3.0,0.0]},
    {'subsample_freq':[0,1,2,5,10,30]},
    {'colsample_bytree':[1.0,2.0,3.0,0.0]},
    {'reg_alpha':[0.0,0.1,1.0,2.0,0.1,0.2]},
    {'reg_lambda':[0.0,0.1,1.0,2.0,0.1,0.2]},
    {'n_jobs':[-1,0,1,10,5]},
    {'importance_type':['split','gain']},
    {'max_bin':[255,10,25,5,100,150]}
]
##instantiation du model
lgbm_best_model=GridSearchCV(lgbm,params,cv=15,verbose=16)
lgbm_best_model.fit(X_train,Y_train)

Fitting 15 folds for each of 92 candidates, totalling 1380 fits
[CV 1/15; 1/92] START boosting_type=gbdt........................................
[CV 1/15; 1/92] END .........boosting_type=gbdt;, score=0.994 total time=   0.3s
[CV 2/15; 1/92] START boosting_type=gbdt........................................
[CV 2/15; 1/92] END .........boosting_type=gbdt;, score=0.998 total time=   0.3s
[CV 3/15; 1/92] START boosting_type=gbdt........................................
[CV 3/15; 1/92] END .........boosting_type=gbdt;, score=0.993 total time=   0.3s
[CV 4/15; 1/92] START boosting_type=gbdt........................................
[CV 4/15; 1/92] END .........boosting_type=gbdt;, score=0.995 total time=   0.3s
[CV 5/15; 1/92] START boosting_type=gbdt........................................
[CV 5/15; 1/92] END .........boosting_type=gbdt;, score=0.997 total time=   0.3s
[CV 6/15; 1/92] START boosting_type=gbdt........................................
[CV 6/15; 1/92] END .........boosting_type=gb

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
150 fits failed out of a total of 1380.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py", line 744, in fit
    callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py", line 544, in fit
    callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py", line 197, in train
    bo

GridSearchCV(cv=15, estimator=LGBMClassifier(),
             param_grid=[{'boosting_type': ['gbdt', 'dart', 'goss', 'rf']},
                         {'num_leaves': [31, 10, 15, 50, 40, 31, 100, 500]},
                         {'max_depth': [-1, 10, 55, 15, 30, 100]},
                         {'learning_rate': [0.1, 0.2, 1.0, 10]},
                         {'n_estimators': [100, 50, 25, 15, 500]},
                         {'subsample_for_bin': [200000, 300000, 500000,
                                                100000]},
                         {'objective': [None, 'binary', 'multiclass']},
                         {'...
                         {'min_child_weight': [0.001, 0.002, 0.1, 0.01, 0.03]},
                         {'min_child_samples': [20, 10, 50, 5, 100]},
                         {'subsample': [1.0, 0.1, 2.0, 1.2, 5.0, 3.0, 0.0]},
                         {'subsample_freq': [0, 1, 2, 5, 10, 30]},
                         {'colsample_bytree': [1.0, 2.0, 3.0, 0.0]},
    

In [27]:
#parametres optionnaux
lgbm_best_model.best_params_

{'min_child_samples': 100}

In [28]:
#model optimal
lgbm_best_model.best_estimator_
lgbm_knn_best = lgbm_best_model.best_estimator_.predict(X_test)

In [29]:
#meilleur score
lgbm_best_model.best_score_

0.9982802200151405

In [30]:
#exporter le modele
import pickle
filename="model_lgbm_sidy.sav"
pickle.dump(lgbm_best_model,open(filename,"wb"))